# HyperParameter Tuning with Elastic Distributed Training example notebook for WMLA

This is a sample notebook showing how to use the "Bring Your Own Framework" API of WMLA to tune community PyTorch
Vision model with HyperParameter Optimization and Elastic Distributed Training of Watson ML Accelerator. 

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

This example uses a Elastic Distributed instance group.

[Kelvin Lui](https://w3.ibm.com/search/#/search?query=Kelvin%20lui)

Based on examples from https://github.ibm.com/kelvinl/WMLA-enablement/ and specifically `WMLA EDT-Copy2`.

BYOF creates execution tasks via API - this is the equivalent of running this command via the `dlicmd` command locally on the cluster. Details of the `dlicmd` command can be found in [dlicmd.py reference](https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html) in Knowledge Centre. 

Note: this setup assumes that models can access data sources local to the WMLA cluster - although this data could be downloaded during the model training phase. 

## Contents:
<a id='WMLA-HPO-Hyperband-Wells-EDT'></a>
- [Log on](#Log-on)
- [Health Check](#Health-Check)
- [Update Model](#Update-model)
- [Launch HPO Task](#Launch-HPO-Task)
- [Check HPO Status](#Check-HPO-Status)
- [Manage HPO tasks](#Manage-HPO-Task)


### Common Setup

For WMLA cluster, URLs required are:
```
$ egosh client view ASCD_REST_BASE_URL_1 | grep DESCRIPTION | cut -d' ' -f2
http://hostname.local:8280/platform/rest/
```
Giving: http://hostname.local:8280/platform/rest/conductor/v1

```
$ egosh client view DLPD_REST_BASE_URL_1 | grep DESCRIPTION | cut -d' ' -f2
http://hostname.local:9243/platform/rest/
```
Giving: http://hostname.local:9243/platform/rest/deeplearning/v1

API documentation links (can also be found on both links above):

- IBM Spectrum Conductor RESTful APIs: http://hostname.local:8280/cloud/apis/explorer/
- IBM Spectrum Conductor Deep Learning Impact RESTful APIs: http://hostname.local:9243/cloud/apis/explorer/
- Also further details on Knowledge Centre: https://www.ibm.com/support/knowledgecenter/en/SSWQ2D_1.2.1/cm/deeplearning.html

In [1]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import json
import time
import urllib
import pandas as pd



In [2]:
# Environment details:



master_host = '**** ADD HERE ****'
dli_rest_port = '9280'
sc_rest_port = '8280'
protocol = 'http'

sc_rest_url =  protocol+'://'+master_host+':'+sc_rest_port+'/platform/rest/conductor/v1'
dl_rest_url = protocol+'://'+master_host+':'+dli_rest_port+'/platform/rest/deeplearning/v1'

print (sc_rest_url)
print (dl_rest_url)

# User login details
wmla_user = '**** ADD HERE ****'
wmla_pwd = '**** ADD HERE ****'


myauth = (wmla_user, wmla_pwd)

# Spark instance group details
sig_name = '**** ADD HERE ****'


# REST call variables
commonHeaders = {'Accept': 'application/json'}


#startTuneUrl='%s://%s:%s/platform/rest/deeplearning/v1/hypersearch' % (protocol, master_host, dli_rest_port)
#sc_rest_url ='%s://%s:%d/platform/rest/conductor/v1' % (protocol, hostname, conductorport)

req = requests.Session()

http://colonia05.platform:8280/platform/rest/conductor/v1
http://colonia05.platform:9280/platform/rest/deeplearning/v1


## Log On
<a id='Log-on'></a>

[back to top](#WMLA-HPO-Hyperband-Wells-EDT)

Obtain login session tokens to be used for session authentication within the RESTful API. Tokens are valid for 8 hours.

In [3]:
r = requests.get(sc_rest_url+'/auth/logon', verify=False, auth=myauth, headers=commonHeaders) 

if r.ok:
    print ('\nLogon succeeded')
    
else: 
    print('\nLogon failed with code={}, {}'. format(r.status_code, r.content))


Logon succeeded


### Health Check
[back to top](#WMLA-HPO-Hyperband-Wells-EDT)



Check if there is any existing hpo tasks and also verify the platform health

Rest API: **GET platform/rest/deeplearning/v1/hypersearch**
- Description: Get all the hpo task that the login user can access.
- OUTPUT: A list of hpo tasks and each one with the same format which can be found in the api doc.

In [4]:
getTuneStatusUrl = dl_rest_url + '/hypersearch'
print ('getTuneStatusUrl: %s' %getTuneStatusUrl)
r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=myauth)

if not r.ok:
    print('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
else:
    if len(r.json()) == 0:
        print('There is no hpo task been created')
    for item in r.json():
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
        #print('Best:%s'%json.dumps(item.get('best'), sort_keys=True, indent=4))

getTuneStatusUrl: http://colonia05.platform:9280/platform/rest/deeplearning/v1/hypersearch
Hpo task: Admin-hpo-213064641612058, State: FAILED
Hpo task: Admin-hpo-213144126479233, State: FAILED
Hpo task: Admin-hpo-213397684489627, State: FAILED
Hpo task: Admin-hpo-217042344463302, State: RECOVERPEND
Hpo task: Admin-hpo-217437872366884, State: RECOVERPEND
Hpo task: Admin-hpo-217638395891962, State: RECOVERPEND
Hpo task: Admin-hpo-1271337322108204, State: FAILED
Hpo task: Admin-hpo-1271754074212915, State: RECOVERPEND
Hpo task: Admin-hpo-1271830782366418, State: FAILED
Hpo task: Admin-hpo-1595647470952563, State: FAILED
Hpo task: Admin-hpo-1598262163401922, State: FAILED
Hpo task: Admin-hpo-1599425793403806, State: FAILED
Hpo task: Admin-hpo-1599772383052641, State: FINISHED
Hpo task: Admin-hpo-1600799291243077, State: FAILED
Hpo task: Admin-hpo-1600966921285655, State: FINISHED
Hpo task: Admin-hpo-1849374454171515, State: FAILED
Hpo task: Admin-hpo-1849606511990094, State: FAILED
Hpo tas

## Update Model 
[back to top](#WMLA-HPO-Hyperband-Wells-EDT)

We will update the model to enable:
 * Elastic Distributed Training
 * Hyper-parameter Optimization 

###  Model file update to run Elastic Distributed Training

Elastic Distributed Training (EDT) takes a model built on a standalone system,  and distributes model training across multiple GPUs and compute nodes in elastic fashion.   EDT starts job execution with minimal 1 GPU,  and GPUs can be added or removed dynamically while executing.

We will use Elastic Distributed Training to distribute a single HPO task with multiple gpus.


In this sample we will update the PyTorch RestNet18 model and enable Elastic Distributed Training: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

Model changes required from 3 perspective:
* Import Elastic Distributed Training library and environment variable
* Replace the data loading functions with ones that are compatible with EDT - the data loading function must return a tuple containing two items of type torch.utils.data.Dataset 
* Replace the training and testing loops with EDT’s train function


##### Model update part 1.  Import Elastic Distributed Training library and environment variable

<pre>
path=os.path.join(os.getenv("FABRIC_HOME"), "libs", "fabric.zip")
print(path)
sys.path.insert(0,path)
from fabric_model import FabricModel
from edtcallback import EDTLoggerCallback

dataDir = environ.get("DATA_DIR")
if dataDir is not None:
    print("dataDir is: %s"%dataDir)
else:
    print("Warning: not found DATA_DIR from os env!")
</pre>

    

##### Model update part 2.  Replace the data loading functions with ones that are compatible with EDT - the data loading function must return a tuple containing two items of type torch.utils.data.Dataset 

* Download the Dataset from https://download.pytorch.org/tutorial/hymenoptera_data.zip

<pre>
def getDatasets():

    data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    }

    return (datasets.ImageFolder(os.path.join(dataDir, 'train'), data_transforms['train']),
           datasets.ImageFolder(os.path.join(dataDir, 'val'), data_transforms['val']))
</pre>         

##### Model update part 3.  Replace the training and testing loops with EDT’s train function

 a. Instantiate Elastic Distributed Training instance
 <pre>
 edt_m = FabricModel(model, getDatasets, loss_function, optimizer, driver_logger=EDTLoggerCallback())
 </pre>
 b. Launch EDT job with specific parameters
 <pre>
 edt_m.train(epoch_number, effective_batch_size, max_number_GPUs_assigned_to_EDT_job)
 </pre>

### Model file update to Run HPO

Model changes required from 2 perspective:
- Inject hyper-parameters for the sub-training during search
- Retrieve sub-training result metric

##### Model update part 1 - Inject hyper-parameters

The hyper-parameters will be supplied in a file called **config.json** with JSON format,located in the current working directory and can be read direcly as the following example snippet.

<pre>
hyper_params = json.loads(open("<b>config.json</b>").read())
learning_rate = float(hyper_params.get("<b>learning_rate</b>", "0.01"))
</pre>

After this, you can use these hyper-parameters during the model trainings. The **hyper-parameter name** and **value** type is defined through the search space part in body of REST call when launching a new hpo task.

##### Model update part 2 - Retrieve sub-training result metric

At the end of your training run, your code will need to create a file called **val_dict_list.json** with test metrics generated during training. These metrics will be used by the search algorithm to propose new sets of hyper-parameters. Please note that **val_dict_list.json** should be created under the result directory which can be retrieved through the environment variable **RESULT_DIR**.

<pre>
with open('{}/val_dict_list.json'.format(os.environ['<b>RESULT_DIR</b>']), 'w') as f:
    json.dump(test_metrics, f)
</pre>

We add this code in the function def on_train_end(self) defined in edtcallback.py  

The content of **val_dict_list.json** will be some thing as below, **step** is some thing optional meaning the training iteration or epochs, one of **loss** and **accuracy** can be the name of target metric to optimize, at least one metric need to be included here. The specific name of metric used to optimize (minimize or maximize) is defined in the body of REST call when launching a new hpo task. 

```
[
{‘step’: 1, ‘loss’:0.2487, ‘accuracy’: 0.4523},
{‘step’: 2, ‘loss’:0.1487, ‘accuracy’: 0.5523},
{‘step’: 3, ‘loss’:0.1087, ‘accuracy’: 0.6523},
…
]
```

## Launch HPO task

REST API: **POST /platform/rest/deeplearning/v1/hypersearch**
- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model files tar package, ending with `.modelDir.tar`
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


##### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

In [5]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
MODEL_DIR_SUFFIX = ".modelDir.tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/Users/Kelvin/Documents/WorkInProgress/WellsFargo/CSSC_EDT/pytorch_edt_cssc2')
files = {'file': open(tempFile, 'rb')}

##### Construct POST request data

**hpo_input** will be a Python dict or json format as below, convert to string when calling REST.

In [6]:

data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': sigName,

            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            # maxWorker is the number of GPUs assigned to single HPO task
            
            # Define parameters passed into the model
            # data_path = 'hymenopteradata' # relative path of your data se
            # 'edtPyTorch' # DL Framework to use, from list given above
            #. model_main = 'pytorch_mnist_EDT.py' # Main model file as opened locally above
            # number_of_GPU = '2' # number of GPUs for elastic distribution
            
            # In this example there are total 12 subtraining (maxJobNum=12).   There are 2 subtraining running 
            # in parallel (maxParalleJobNum=2).  Each subtraining is distributed across 2 GPUs with Elastic 
            # Distributed Training
            #

            
            'args': '--model-dir pytorch_edt_cssc2 --exec-start edtPyTorch  \
                     --cs-datastore-meta type=fs,data_path=hymenoptera_data \
                     --edt-options maxWorkers=2 --model-main pytorch_mnist_HPO_EDT.py \
                     --debug-level debug  \
                    '

        },    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband
            'algorithm': 'Hyperband', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': -1,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 12,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 2, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
            # eta value to control the hyper-band search process
            'hyperbandEta': 3.0,
            #Additional parameters for the specified search algorithm and hyper-band get following too.
            'algoParams' : 
                [
                    {
                        # Name of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceName', 
                        'value': 'epochs'
                    },
                    {
                        # Value of the the maximum amount of resource that can be allocated to a single configuration
                        'name':'ResourceValue',
                        'value':'81'
                    }
                    # This resource parameter will change during hyperband searching phase and its value will be put
                    # into the config.json for each sub-training too, here with the 'epoch' as key and value in the
                    # range 1-10.
                ]
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': [],
                 #step size to split the Range space. ONLY valid when type is Range
                 'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

##### Submit the Post request

Submit hpo task through the Post call and a hpo name/id as string format will get back.

In [7]:
startTuneUrl=dl_rest_url + '/hypersearch'
r = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=myauth)
if r.ok:
    hpoName = r.json()
    json_out_1 = r.json()
    print ('HPO Application: %s' %hpoName)

else: 
    print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))

HPO Application: Admin-hpo-2540782107139805


In [8]:
import time

getHpoUrl = dl_rest_url +'/hypersearch/'+ hpoName

res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
else:
    json_out=res.json()
    
    while json_out['state'] in ['SUBMITTED','RUNNING']:
        print('Hpo task %s state %s progress %s%%'%(hpoName, json_out['state'], json_out['progress']))
        time.sleep(90)
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
        json_out=res.json()
        
        experiments_length = len(json_out['experiments'])
       
        ####
        ## Query the list of 6 sub-training of current batch, as maxParalleJobNum=6
        ###      
        count=0
        Experiment = []
        while (count < experiments_length):
                if('appId' in json_out['experiments'][count]):
                    appID = json_out['experiments'][count]['appId']
                    #print ('appID: %s,' %appID )
                    #print ('count: %d' %count)
                    Experiment.insert(count, appID)
                count+=1
 
        ####
        ## Query the state of 6 sub-training of current batch
        ###
    
        count = 0
        while (count < len(Experiment)):
                r = requests.get(dl_rest_url+'/execs/'+Experiment[count], auth=myauth, headers=commonHeaders, verify=False).json()    
                if not res.ok:
                    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
                else:
                    print ('Experiement %s state: %s' %(Experiment[count], r['state']))
                count+=1
        
        #time.sleep(30)
        #print ('state:' + json_out['state'] )

        
print('Hpo task %s completes with state %s'%(hpoName, json_out['state']))
print(json.dumps(json_out, indent=4, sort_keys=True))
 

Hpo task Admin-hpo-2540782107139805 state RUNNING progress 0/12%
Experiement Admin-2540783202199178-1008599601 state: FINISHED
Experiement Admin-2540789124256407-1015451251 state: FINISHED
Experiement Admin-2540857169102214-1860714260 state: RUNNING
Experiement Admin-2540863072099183-1342712267 state: RUNNING
Hpo task Admin-hpo-2540782107139805 state RUNNING progress 2/12%
Experiement Admin-2540783202199178-1008599601 state: FINISHED
Experiement Admin-2540789124256407-1015451251 state: FINISHED
Experiement Admin-2540857169102214-1860714260 state: FINISHED
Experiement Admin-2540863072099183-1342712267 state: FINISHED
Experiement Admin-2540931249625623-36562925 state: FINISHED
Experiement Admin-2540937239191104-1810778475 state: RUNNING
Hpo task Admin-hpo-2540782107139805 state RUNNING progress 4/12%
Experiement Admin-2540783202199178-1008599601 state: FINISHED
Experiement Admin-2540789124256407-1015451251 state: FINISHED
Experiement Admin-2540857169102214-1860714260 state: FINISHED
Expe

**Full Response of a hpo task status.**

```
{
"hpoName": "string, name/id of the hpo task",
"state": "string, hpo task state, SUBMITTED, RUNNING, FAILED, FINISHED, STOPPED",
"running": "int, number of training that is under-going right now",
"complete": "int, number of training that has completes, including both succeeded and failed trainings",
"failed": "int, number of training that failed",
"progress": "string, progress of a percentage value",
"createtime": "string, time this task was created",
"creator": "string, user name who created this hpo task",
"duration": "string, how long the task been run with format hh-mm-ss",
"experiments": [
                {
                "id": "int, counter id of the experiment training, start from 0",
                "metricVal": "double, best metric value of the experiment training",
                "state": "string, state of the BYOF training task"
                "appId": "string, BYOF training task id",
                "driverId": "string, for internal usage, the real job id for training, currently it is the spark driver id",
                "hyperParams": [
                                {
                                "name": "string, name of the hyperparameter",
                                "dataType": "string, data type of the hyperparameter",
                                "fixedVal": "string, The hyperparamter value been used in this experiment training with the same datatype as input dataType"
                                }
                                ],
                }
                ],
"best": {"one of the experiments with the best metric value, smallest or biggest one based on the original objective minimize or maximize"}
}

```
